In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import locale
import os
locale.setlocale(locale.LC_ALL,'pt_BR.UTF-8')
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.cluster import KMeans
import statsmodels.api as sm
from scipy.stats import f_oneway
import gc
gc.collect()
from pathlib import Path

: 

## Início do modelo



In [ ]:
# -----------------------------
# 1) Diretórios básica
# -----------------------------
DATA_DIR = Path(r"Data\parquet") 
OUT_DIR  = Path(r"Output")
OUT_DIR.mkdir(parents=True, exist_ok=True)


# Lista de arquivos
arquivos = {
    # centrais
    "dados_clientes": DATA_DIR / "dados_clientes.parquet",
    "clientes_desde": DATA_DIR / "clientes_desde.parquet",
    "contratacoes_12m": DATA_DIR / "contratacoes_ultimos_12_meses.parquet",
    "historico": DATA_DIR / "historico.parquet",
    "mrr": DATA_DIR / "mrr.parquet",
    "tickets": DATA_DIR / "tickets.parquet",
    # NPS
    "nps_relacional": DATA_DIR / "nps_relacional.parquet",
    "nps_tx_aquisicao": DATA_DIR / "nps_transacional_aquisicao.parquet",
    "nps_tx_implantacao": DATA_DIR / "nps_transacional_implantacao.parquet",
    "nps_tx_onboarding": DATA_DIR / "nps_transacional_onboarding.parquet",
    "nps_tx_produto": DATA_DIR / "nps_transacional_produto.parquet",
    "nps_tx_suporte": DATA_DIR / "nps_transacional_suporte.parquet",
    # telemetria
    **{f"telemetria_{i}": DATA_DIR / f"telemetria_{i}.parquet" for i in range(1, 12)}
}

def ler_parquet(p: Path):
    try:
        return pd.read_parquet(p)
    except Exception:
        return None

def normalizar_colunas(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = [c.strip().replace(" ", "_").replace("-", "_").replace(".", "_").lower() for c in df.columns]
    return df

gc.collect()

# -------------------------------------------
# 2) Leitura e união por família de dataframe
# -------------------------------------------
tabelas = {}
for nome, caminho in arquivos.items():
    df = ler_parquet(caminho)
    if df is not None and len(df.columns) > 0:
        tabelas[nome] = normalizar_colunas(df)

# Une telemetria
telemetria = pd.concat([df for k, df in tabelas.items() if k.startswith("telemetria_")], ignore_index=True) \
             if any(k.startswith("telemetria_") for k in tabelas) else None

# Une NPS transacional
nps_trans = pd.concat([tabelas[k] for k in tabelas if k.startswith("nps_tx_")], ignore_index=True) \
           if any(k.startswith("nps_tx_") for k in tabelas) else None

# NPS relacional
nps_rel = tabelas.get("nps_relacional")

# Outras bases
dados_clientes   = tabelas.get("dados_clientes")
clientes_desde   = tabelas.get("clientes_desde")
contratacoes_12m = tabelas.get("contratacoes_12m")
historico        = tabelas.get("historico")
mrr              = tabelas.get("mrr")
tickets          = tabelas.get("tickets")

gc.collect()

# -----------------------------
# 3) Achando colunas-chave (simples)
# -----------------------------
# Tentamos nomes comuns para CID, DATA, VALOR — se não achar, usamos casamento simples de nome de colunas
def coluna_cliente(df):
    if df is None: return None
    for c in ["cod_cliente", "cd_cliente", "metadata_codcliente", "cid", "cliente_id", "id_cliente"]:
        if c in df.columns: return c

    for c in df.columns:
        if "cliente" in c or c.endswith("cid"):
            return c
    return None

def coluna_data(df):
    if df is None: return None
    for c in ["purchase_date","respondedat","responded_at","data","dt_evento","created_at","dt","mes","competencia"]:
        if c in df.columns: return c
    for c in df.columns:
        if "date" in c or "data" in c or c.startswith("dt"):
            return c
    return None

def coluna_valor(df):
    if df is None: return None
    for c in ["net_amount","valor","vlr","mrr","valor_total","amount","gross_amount"]:
        if c in df.columns: return c
    for c in df.columns:
        if ("valor" in c) or ("amount" in c) or ("mrr" in c):
            return c
    return None

gc.collect()

# -----------------------------
# 4) Monta EVENTS (para recency/frequency)
# -----------------------------
def montar_events(lista_df):
    frames = []
    for nome, df in lista_df:
        if df is None: 
            continue
        cid = coluna_cliente(df)
        dt  = coluna_data(df)
        if cid and dt and cid in df.columns and dt in df.columns:
            tmp = df[[cid, dt]].copy()
            tmp.columns = ["cid", "event_dt"]
            tmp["cid"] = tmp["cid"].astype(str)
            tmp["event_dt"] = pd.to_datetime(tmp["event_dt"], errors="coerce")
            tmp = tmp.dropna(subset=["cid", "event_dt"])
            frames.append(tmp)
    if frames:
        return pd.concat(frames, ignore_index=True)
    else:
        return pd.DataFrame(columns=["cid", "event_dt"])

events = montar_events([
    ("nps_rel", nps_rel),
    ("nps_trans", nps_trans),
    ("historico", historico),
    ("mrr", mrr),
    ("telemetria", telemetria),
    ("tickets", tickets)
])

gc.collect()

# -----------------------------
# 5) Monta MONETARY (somas por cliente)
# -----------------------------
def montar_monetary(lista_df):
    frames = []
    for nome, df in lista_df:
        if df is None:
            continue
        cid = coluna_cliente(df)
        val = coluna_valor(df)
        if cid and val and cid in df.columns and val in df.columns:
            tmp = df[[cid, val]].copy()
            tmp.columns = ["cid", "amount"]
            tmp["cid"]   = tmp["cid"].astype(str)
            tmp["amount"] = pd.to_numeric(tmp["amount"], errors="coerce")
            frames.append(tmp)
    if frames:
        out = pd.concat(frames, ignore_index=True)
        out["amount"] = out["amount"].fillna(0.0)
        return out
    else:
        return pd.DataFrame(columns=["cid", "amount"])

monetary = montar_monetary([
    ("mrr", mrr),
    ("contratacoes_12m", contratacoes_12m),
    ("historico", historico)
])

gc.collect()

# -----------------------------
# 6) Calcula RFM (Recency, Frequency, Monetary):
# -----------------------------
# Identifica comportamento, último evento, frequência e valores
if len(events) > 0:
    ref_date = events["event_dt"].max() + pd.Timedelta(days=1)
    last_evt = events.groupby("cid", as_index=False)["event_dt"].max().rename(columns={"event_dt":"last_event"})
    freq     = events.groupby("cid", as_index=False).size().rename(columns={"size":"frequency"})
    rfm = last_evt.merge(freq, on="cid", how="outer")
else:
    ref_date = pd.Timestamp.today()
    rfm = pd.DataFrame(columns=["cid","last_event","frequency"])

if len(monetary) > 0:
    money = monetary.groupby("cid", as_index=False)["amount"].sum().rename(columns={"amount":"monetary"})
    rfm = rfm.merge(money, on="cid", how="left")
else:
    rfm["monetary"] = 0.0

rfm["last_event"] = pd.to_datetime(rfm.get("last_event"), errors="coerce")
rfm["recency"]    = (ref_date - rfm["last_event"]).dt.days
rfm["frequency"]  = pd.to_numeric(rfm.get("frequency"), errors="coerce").fillna(0).astype(int)
rfm["monetary"]   = pd.to_numeric(rfm.get("monetary"), errors="coerce").fillna(0.0)


rfm.to_csv(OUT_DIR / "rfm_raw.csv", index=False)

rfm_model = rfm.dropna(subset=["recency"]).copy()

gc.collect()

# -----------------------------
# 7) KMeans (k=3) – simples
# -----------------------------
if len(rfm_model) >= 3:
    # log1p para suavizar, alterado pelo Helder
    X = np.log1p(rfm_model[["recency","frequency","monetary"]].values)   
    X = StandardScaler().fit_transform(X)                               
    km = KMeans(n_clusters=3, random_state=42, n_init=10).fit(X)
    rfm_model["cluster"] = km.labels_


    profiles = (rfm_model.groupby("cluster")
                .agg(recency_mean=("recency","mean"),
                     frequency_mean=("frequency","mean"),
                     monetary_mean=("monetary","mean"),
                     customers=("cid","nunique"))
                .reset_index()
                .sort_values(["recency_mean","frequency_mean","monetary_mean"],
                             ascending=[True, False, False]))

    # rótulos mais amigáveis (colunas semelhantes feitas pela Liora)
    ordem = profiles["cluster"].tolist()
    nomes = ["Best","Loyal","At Risk"]  # se k=3
    mapa = {cl: nome for cl, nome in zip(ordem, nomes)}
    rfm_model["segment"] = rfm_model["cluster"].map(mapa)
else:
    profiles = pd.DataFrame(columns=["cluster","recency_mean","frequency_mean","monetary_mean","customers"])
    rfm_model["cluster"] = np.nan
    rfm_model["segment"] = np.nan

rfm_model.to_csv(OUT_DIR / "rfm_with_clusters.csv", index=False)
profiles.to_csv(OUT_DIR / "cluster_profiles.csv", index=False)

gc.collect()

# --------------------------------------
# 8) Tendência mensal de eventos + flag de alerta
# --------------------------------------

if len(events) > 0:
    events = events.copy()
    events['event_dt'] = pd.to_datetime(events['event_dt'], errors='coerce')
    events = events.dropna(subset=['event_dt'])


    # mask = (events['event_dt'] >= '2024-01-01') & (events['event_dt'] < '2025-01-01') ## não esta muito bom isso ainda
    # events = events.loc[mask]

    # Agrega por mês-calendário (MS = Month Start) e normaliza para "1º dia do mês"
    monthly = (
        events.set_index('event_dt')
              .groupby(pd.Grouper(freq='MS'))
              .agg(events=('cid', 'count'))
              .reset_index()
              .rename(columns={'event_dt': 'month'})
    )
    monthly['month'] = pd.to_datetime(monthly['month'], errors='coerce')
    monthly = monthly.dropna(subset=['month']).sort_values('month')
    monthly['month'] = monthly['month'].dt.to_period('M').dt.to_timestamp()

    # janela 6 meses
    monthly['ma'] = monthly['events'].rolling(6, min_periods=3).mean()
    monthly['sd'] = monthly['events'].rolling(6, min_periods=3).std()
    monthly['lo_band'] = monthly['ma'] - 2 * monthly['sd']
    monthly['anomaly_low'] = monthly['events'] < monthly['lo_band']

else:
    monthly = pd.DataFrame(columns=['month', 'events', 'ma', 'sd', 'lo_band', 'anomaly_low'])

monthly.to_csv(OUT_DIR / 'monthly_events_trend.csv', index=False)


gc.collect()

# -----------
# 9) Gráficos 
# -----------
# 9.1 média monetária por cluster
if len(profiles) > 0:
    plt.figure(figsize=(8,4))
    plt.bar(profiles["cluster"].astype(str), profiles["monetary_mean"])
    plt.title("Cluster monetary mean")
    plt.xlabel("Cluster"); plt.ylabel("Monetary (mean)")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "chart_cluster_monetary_mean.png", dpi=150)
    plt.close()

# 9.2 dispersão Frequency x Monetary
if "cluster" in rfm_model.columns and rfm_model["cluster"].notna().any():
    plt.figure(figsize=(7,5))
    plt.scatter(np.log1p(rfm_model["frequency"]), np.log1p(rfm_model["monetary"]))
    plt.title("Log(Frequency) vs Log(Monetary)")
    plt.xlabel("log(Frequency)")
    plt.ylabel("log(Monetary)")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "chart_scatter_freq_monetary.png", dpi=150)
    plt.close()

# 9.3 tendência mensal de eventos
if len(monthly) > 0:
    plt.figure(figsize=(10,4))
    plt.plot(monthly["event_dt"], monthly["events"])
    plt.title("Monthly Events Trend")
    plt.xlabel("Month"); 
    plt.ylabel("Events")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "chart_monthly_events_trend.png", dpi=150)
    plt.close()


# -----------------------------
# 10) Resumo executivo (pequeno)
# -----------------------------
exec_summary = pd.DataFrame({
    "Metric": [
        "Customers (RFM table)",
        "Customers modeled",
        "Total monetary (sum)",
        "Best segment size",
        "Months with low anomaly"
    ],
    "Value": [
        rfm["cid"].nunique() if "cid" in rfm.columns else 0,
        rfm_model["cid"].nunique() if "cid" in rfm_model.columns else 0,
        float(np.nansum(rfm_model["monetary"])) if "monetary" in rfm_model.columns else 0.0,
        int((rfm_model["segment"] == "Best").sum()) if "segment" in rfm_model.columns else 0,
        int(monthly["anomaly_low"].sum()) if "anomaly_low" in monthly.columns else 0
    ]
})
exec_summary.to_csv(OUT_DIR / "executive_summary.csv", index=False)

gc.collect()

: 

## Testes de Hipótese


In [ ]:
# === Testes de Hipótese: ANOVA + Tukey entre clusters (monetary) ===

OUT_DIR = Path("Output")
OUT_DIR.mkdir(exist_ok=True, parents=True)

# Remove NaN/inf
df_th = rfm_model[['cluster','monetary']].dropna().copy()
df_th = df_th[np.isfinite(df_th['monetary'])]

# ANOVA (médias de 'monetary' entre clusters)
grupos = [g['monetary'].values for _, g in df_th.groupby('cluster')]
anova_F, anova_p = f_oneway(*grupos) if len(grupos) > 1 else (np.nan, np.nan)

# Tukey HSD (pareado)
# Para Tukey, precisamos de vetor de valores e vetor de grupos
if df_th['cluster'].nunique() > 1 and len(df_th) > 0:
    tukey = pairwise_tukeyhsd(endog=df_th['monetary'].values,
                              groups=df_th['cluster'].astype(str).values,
                              alpha=0.05)
    tukey_result = pd.DataFrame(tukey.summary().data[1:], columns=tukey.summary().data[0])
else:
    tukey_result = pd.DataFrame(columns=['group1','group2','meandiff','p-adj','lower','upper','reject'])

anova_out = pd.DataFrame({
    'metric': ['monetary'],
    'F': [anova_F],
    'p_value': [anova_p]
})
anova_out.to_csv(OUT_DIR / "anova_results.csv", index=False)

tukey_result.to_csv(OUT_DIR / "tukey_results.csv", index=False)

print("ANOVA (monetary) -> F =", round(anova_F,3), "| p-value =", round(anova_p,6))
print("Arquivos salvos:")
print(" - Output/anova_results.csv")
print(" - Output/tukey_results.csv")

ANOVA (monetary) -> F = 753.477 | p-value = 0.0
Arquivos salvos:
 - Output/anova_results.csv
 - Output/tukey_results.csv


## Previsão M+1

In [ ]:
OUT_DIR = Path("Output")
OUT_DIR.mkdir(exist_ok=True, parents=True)

# Carrega a série mensal 
monthly_path = OUT_DIR / "monthly_events_trend.csv"
if monthly_path.exists():
    monthly_pred = pd.read_csv(monthly_path, parse_dates=['event_dt'], sep=',')
    
elif 'monthly' in globals():
    monthly_pred = monthly.copy()
else:
    monthly_pred = pd.DataFrame(columns=['month','events'])
    print("Aviso: monthly_events_trend.csv não encontrado e 'monthly' não existe. Pulo a previsão.")

if not monthly_pred.empty and 'events' in monthly_pred.columns:
    # Remove nulos
    monthly_pred = monthly_pred[['event_dt','events']].dropna().sort_values('event_dt').reset_index(drop=True)
    monthly_pred['t'] = np.arange(len(monthly_pred)) 

    # Modelo 1: Naive 
    naive_forecast = monthly_pred['events'].iloc[-1] if len(monthly_pred) > 0 else np.nan

    # Modelo 2: Regressão Linear 
    lr = LinearRegression()
    X = monthly_pred[['t']].values
    y = monthly_pred['events'].values
    if len(monthly_pred) >= 2 and np.isfinite(y).all():
        lr.fit(X, y)
        next_t = np.array([[monthly_pred['t'].iloc[-1] + 1]])
        lr_forecast = float(lr.predict(next_t)[0])
    else:
        lr_forecast = np.nan

    # Próximo mês 
    if len(monthly_pred) > 0:
        next_month = (monthly_pred['event_dt'].iloc[-1] + pd.offsets.MonthBegin(1)).normalize()
    else:
        next_month = pd.NaT

    preds = pd.DataFrame({
        'next_month': [next_month],
        'naive_events_forecast': [naive_forecast],
        'lr_events_forecast': [lr_forecast]
    })

    preds.to_csv(OUT_DIR / "predictions_m_plus_1.csv", index=False)
    print("Previsões salvas em: Output/predictions_m_plus_1.csv")
else:
    print("Sem dados mensais de events para prever M+1.")


: 